# `pipestat` CLI

Before following this tutorial please make sure you're familiar with "`pipestat` Python API" tutorial.

## Usage reference

To learn about the usage `pipestat` usage use `--help`/`-h` option on any level:

In [1]:
pipestat -h

version: 0.0.1
usage: pipestat [-h] [--version] [--silent] [--verbosity V] [--logdev]
                {report,inspect,remove} ...

pipestat - report pipeline results

positional arguments:
  {report,inspect,remove}
    report              Report a result.
    inspect             Inspect a database.
    remove              Remove a result.

optional arguments:
  -h, --help            show this help message and exit
  --version             show program's version number and exit
  --silent              Silence logging. Overrides verbosity.
  --verbosity V         Set logging level (1-5 or logging module level name)
  --logdev              Expand content of logging message format.

Pipestat provides methods for a standardized reporting of pipeline statistics.
It formalizes a way for pipeline developers and downstream tools developers to
communicate -- results produced by a pipeline can easily and reliably become
an input for downstream analyses.


In [2]:
pipestat report -h

usage: pipestat report [-h] -n N (-f F | -c C) -s S -i I -r R -v V [-o] [-t]

Report a result.

optional arguments:
  -h, --help            show this help message and exit
  -n N, --namespace N   Name of the pipeline to report result for
  -f F, --results-file F
                        Path to the YAML file where the results will be
                        stored. This file will be used as pipestat backend and
                        to restore the reported results across sesssions
  -c C, --database-config C
                        Path to the YAML file with PostgreSQL database
                        configuration. Please refer to the documentation for
                        the file format requirements.
  -s S, --schema S      Path to the schema that defines the results that can
                        be reported
  -i I, --result-identifier I
                        ID of the result to report; needs to be defined in the
                        schema
  -r R, --record-identifier R


In [3]:
pipestat remove -h

usage: pipestat remove [-h] -n N (-f F | -c C) -s S -i I -r R

Remove a result.

optional arguments:
  -h, --help            show this help message and exit
  -n N, --namespace N   Name of the pipeline to report result for
  -f F, --results-file F
                        Path to the YAML file where the results will be
                        stored. This file will be used as pipestat backend and
                        to restore the reported results across sesssions
  -c C, --database-config C
                        Path to the YAML file with PostgreSQL database
                        configuration. Please refer to the documentation for
                        the file format requirements.
  -s S, --schema S      Path to the schema that defines the results that can
                        be reported
  -i I, --result-identifier I
                        ID of the result to report; needs to be defined in the
                        schema
  -r R, --record-identifier R
               

In [4]:
pipestat inspect -h

usage: pipestat inspect [-h] -n N (-f F | -c C) -s S [-d]

Inspect a database.

optional arguments:
  -h, --help            show this help message and exit
  -n N, --namespace N   Name of the pipeline to report result for
  -f F, --results-file F
                        Path to the YAML file where the results will be
                        stored. This file will be used as pipestat backend and
                        to restore the reported results across sesssions
  -c C, --database-config C
                        Path to the YAML file with PostgreSQL database
                        configuration. Please refer to the documentation for
                        the file format requirements.
  -s S, --schema S      Path to the schema that defines the results that can
                        be reported
  -d, --data            Whether to display the data


## Usage demonstration

### Reporting

Naturally, the command line interface provides access to all the Python API functionalities of `pipestat`. So, for example, to report a result and back the object by a file use:

In [14]:
temp_file=`mktemp`
pipestat report -f $temp_file -n test -r sample1 -i number_of_things -v 100 -s ../tests/data/sample_output_schema.yaml --try-convert

Reading data from '/var/folders/3f/0wj7rs2144l9zsgxd3jn5nxc0000gn/T/tmp.ispNtLl9'
Reported record for 'sample1': number_of_things=100 in 'test' namespace


The result has been reported and the database file has been updated:

In [15]:
cat $temp_file

test:
  sample1:
    number_of_things: 100


Let's report another result:

In [16]:
pipestat report -f $temp_file -n test -r sample1 -i percentage_of_things -v 1.1 -s ../tests/data/sample_output_schema.yaml --try-convert

Reading data from '/var/folders/3f/0wj7rs2144l9zsgxd3jn5nxc0000gn/T/tmp.ispNtLl9'
Reported record for 'sample1': percentage_of_things=1.1 in 'test' namespace


In [17]:
cat $temp_file

test:
  sample1:
    number_of_things: 100
    percentage_of_things: 1.1


### Inspection

In [21]:
pipestat inspect -d $temp_file -n test -s ../tests/data/sample_output_schema.yaml -f $temp_file

usage: pipestat inspect [-h] -n N (-f F | -c C) -s S [-d]
pipestat inspect: error: one of the arguments -f/--results-file -c/--database-config is required


: 2

In [20]:
pipestat inspect -d $temp_file -n test -s ../tests/data/sample_output_schema.yaml -f $temp_file

usage: pipestat inspect [-h] -n N (-f F | -c C) -s S [-d]
pipestat inspect: error: one of the arguments -f/--results-file -c/--database-config is required


: 2

### Removal

In [10]:
pipestat remove -d $temp_file -n test -i files_size

In [11]:
cat $temp_file

test:
  files_count:
    value: '100'
    type: integer


In [12]:
pipestat inspect -d $temp_file -n test

PipeStatManager (test)
Database length: 1
Database: {'test': {'files_count': {'value': '100', 'type': 'integer'}}}
Cache length: 0
Cache: {}
